In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.8.0
Running on TPU  ['10.76.96.122:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.76.96.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.76.96.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# **Import Required Libraries**

In [41]:
!pip install pmdarima

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axisartist.axislines import Subplot
from copy import copy, deepcopy
import pytz
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy.stats import boxcox
from pickle import dump, load

from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore')

import joblib

from matplotlib.dates import num2date
from matplotlib.ticker import Formatter
from datetime import datetime

import tensorflow as tf
from tensorflow import GradientTape
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.metrics import RootMeanSquaredError, Mean
from tensorflow.keras.layers import Input, Dense, LSTM, Layer, Bidirectional, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import huber, mse, Loss, Reduction
import tensorflow.keras.backend as K

For Formatting Console Print Commands - ANSI Escape Code

In [43]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# **Import Data**

In [44]:
scaler = load(open('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/data/scaler.pkl', 'rb'))
officeTS = load(open('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/data/officeTS.pkl', 'rb'))

trainSplit = load(open('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/data/trainSplit.pkl', 'rb'))
testSplit = load(open('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/data/testSplit.pkl', 'rb'))
dateTrainSplit = load(open('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/data/dateTrainSplit.pkl', 'rb'))
dateTestSplit = load(open('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/data/dateTestSplit.pkl', 'rb'))

In [45]:
# Perform Standardization
trainScaled = np.transpose(scaler.transform(np.transpose(trainSplit)))
testScaled = np.transpose(scaler.transform(np.transpose(testSplit)))

print(trainScaled.shape)
print(testScaled.shape)

(11, 46760)
(11, 11704)


# **Utility Functions**

In [46]:
# Make Windows
def makeWindowsFromContinuousSteps(officesData, dateData, history, leadSteps):
    '''
    officesData     : 2-D Numpy Array (number of offices, total number of readings)
    dateData        : 2-D Numpy Array (number of offices, total number of readings)
    history         : Number of TIMESTEPS to be considered as Input to Forecasting Model
    leadSteps       : Number of TIMESTEPS to be considered as Output to Forecasting Model
    '''
    dataX, dataY, officeNumber = [], [], []
    dateX, dateY = [], []
    idxWindows = []

    # If only 1 office data is present, i.e. shape of officesData is
    # (total number of readings,), then reshape to (1, total number of readings)
    if len(officesData.shape)==1:
        officesData = officesData.reshape(1,-1)
    elif not len(officesData.shape)==2:
        raise('Unacceptable input shape of officesData variable - ' + str(officesData.shape))
    
    # If only 1 date data is present, i.e. shape of dateData is
    # (total number of readings,), then reshape to (1, total number of readings)
    if len(dateData.shape)==1:
        dateData = dateData.reshape(1,-1)
    elif not len(dateData.shape)==2:
        raise('Unacceptable input shape of dateData variable - ' + str(dateData.shape))

    for i in range(officesData.shape[0]):
        continuousSubSequenceStartIndex = 0
        continuousSubSequenceStopIndex  = 0
        searchingForNAN = True
        seqCounter = 0
        while seqCounter<len(officesData[i]):
            if seqCounter==len(officesData[i])-1:
                if np.isnan(officesData[i, seqCounter]):
                    continuousSubSequenceStopIndex = seqCounter - 1
                else:
                    continuousSubSequenceStopIndex = seqCounter
                ### Perform continuousSubSequence Operation - Extract Windows ###
                if continuousSubSequenceStopIndex - continuousSubSequenceStartIndex + 1 >= history + leadSteps:
                    for k in range(continuousSubSequenceStartIndex, continuousSubSequenceStopIndex+1):
                        if k + history + leadSteps <= continuousSubSequenceStopIndex + 1:
                            dataX.append(officesData[i, k:k+history])
                            dataY.append(officesData[i, k+history:k+history+leadSteps])
                            officeNumber.append(i)
                            dateX.append(dateData[i, k:k+history])
                            dateY.append(dateData[i, k+history:k+history+leadSteps])
                            idxWindows.append((i, k))
                break
            if not np.isnan(officesData[i, seqCounter]):
                if not searchingForNAN:
                    searchingForNAN = True
                    continuousSubSequenceStartIndex = seqCounter
            else:
                if searchingForNAN:
                    searchingForNAN = False
                    continuousSubSequenceStopIndex = seqCounter - 1
                    ### Perform continuousSubSequence Operation - Extract Windows ###
                    if continuousSubSequenceStopIndex - continuousSubSequenceStartIndex + 1 >= history + leadSteps:
                        for k in range(continuousSubSequenceStartIndex, continuousSubSequenceStopIndex+1):
                            if k + history + leadSteps <= continuousSubSequenceStopIndex + 1:
                                dataX.append(officesData[i, k:k+history])
                                dataY.append(officesData[i, k+history:k+history+leadSteps])
                                officeNumber.append(i)
                                dateX.append(dateData[i, k:k+history])
                                dateY.append(dateData[i, k+history:k+history+leadSteps])
                                idxWindows.append((i, k))
            seqCounter += 1
    return dataX, dataY, officeNumber, dateX, dateY, idxWindows

In [47]:
def calcInvScaledError(scaledPrediction, scaledTruth, errorMetric='RMSE', scaler=None, dataToScalerIndices=None):
    if scaler is not None:
        if dataToScalerIndices is None:
            if len(scaler.mean_) > 1:
                raise("Different scaler used for different datapoints but back reference is not provided - dataToScalerIndices is None!")
            else:
                inv_scaledPrediction = np.transpose(scaler.inverse_transform(np.transpose(scaledPrediction)))
                inv_scaledTruth = np.transpose(scaler.inverse_transform(np.transpose(scaledTruth)))
        else:
            inv_scaledPrediction = deepcopy(scaledPrediction)
            inv_scaledTruth = deepcopy(scaledTruth)
            for i in range(len(scaledPrediction)):
                inv_scaledPrediction[i,:] = (scaledPrediction[i,:]*(scaler.mean_[dataToScalerIndices[i]]))+np.sqrt(scaler.var_[dataToScalerIndices[i]])
                inv_scaledTruth[i,:] = (scaledTruth[i,:]*(scaler.mean_[dataToScalerIndices[i]]))+np.sqrt(scaler.var_[dataToScalerIndices[i]])
        prediction = inv_scaledPrediction
        truth = inv_scaledTruth
    else:
        prediction = scaledPrediction
        truth = scaledTruth
    if errorMetric=='RMSE':
        error = mean_squared_error(truth, prediction, squared=False)
    elif errorMetric=='MSE':
        error = mean_squared_error(truth, prediction, squared=True)
    else:
        raise('Unsupported error metric - ' + errorMetric)
    return error

In [48]:
# Inverse Transform the Standardization Process
def invertStandardization(scaler, data, officeNumber):
    '''
    # Params::
    scaler          : sklearn StandardScaler object which was used to perform standardization
    data            : ndarray of shape (number of windows, time-steps)
    officeNumber    : ndarray of shape (number of windows) - specifying the office number to which a window belongs

    # Returns::
    data_tr         : ndarray of shape (number of windows, time-steps)
    '''
    mean = scaler.mean_
    std  = np.sqrt(scaler.var_)
    data_tr = np.zeros((data.shape))
    for i in range(data.shape[0]):
        data_tr[i,:] = (data[i,:]*std[officeNumber[i]]) + mean[officeNumber[i]]
    return data_tr

# **Main Code Starts Here**

In [49]:
# Selected Working Hours: start_time - end_time (MON-FRI)
start_time = 5
end_time = 19

# Calculate total number of days and readings per day
readingsPerDay = (end_time-start_time)*4
totalDays = int(officeTS.shape[1]/readingsPerDay)
print('Readings Per Day = ', readingsPerDay)
print('Total Number of Days = ', totalDays)

# History and Lead Time
historySteps = 5*readingsPerDay #== 5 Days
leadSteps = 1*readingsPerDay #== 1 Day

Readings Per Day =  56
Total Number of Days =  1044


In [90]:
officeIDX = 1

trainX, trainY, officeNumberTrain, dateTrainX, dateTrainY, idxWindows = makeWindowsFromContinuousSteps(trainScaled[officeIDX], dateTrainSplit[officeIDX], historySteps, leadSteps)
print(len(trainX), len(trainY), len(officeNumberTrain), officeNumberTrain[0])
print(trainX[0].shape, trainY[0].shape)
testX, testY, officeNumberTest, dateTestX, dateTestY, idxWindows = makeWindowsFromContinuousSteps(testScaled[officeIDX], dateTestSplit[officeIDX], historySteps, leadSteps)
print(len(testX), len(testY), len(officeNumberTest), officeNumberTest[0])
print(testX[0].shape, testY[0].shape)

46425 46425 46425 0
(280,) (56,)
10978 10978 10978 0
(280,) (56,)


**Prepare imported data for LSTM/BiLSTM compatibility**

In [91]:
myTrainX = np.array(trainX)
myTrainY = np.array(trainY)
myTestX = np.array(testX)
myTestY = np.array(testY)
print("Before Reshaping")
print(myTrainX.shape, myTrainY.shape)
print(myTestX.shape, myTestY.shape)

# Reshape Input to be [Samples, Time-Steps, Features]
myTrainX = np.reshape(myTrainX, (myTrainX.shape[0], myTrainX.shape[1], 1))
myTestX = np.reshape(myTestX, (myTestX.shape[0], myTestX.shape[1], 1))
print("After Reshaping")
print(myTrainX.shape, myTrainY.shape)
print(myTestX.shape, myTestY.shape)

Before Reshaping
(46425, 280) (46425, 56)
(10978, 280) (10978, 56)
After Reshaping
(46425, 280, 1) (46425, 56)
(10978, 280, 1) (10978, 56)


**Load Models**

In [92]:
BiLSTM_model = Sequential()
BiLSTM_model.add(Input(shape=(myTrainX.shape[1], myTrainX.shape[2])))
forward_layer1 = LSTM(14, activation='tanh', return_sequences=True)
forward_layer2 = LSTM(14, activation='tanh', return_sequences=False)
backward_layer1 = LSTM(14, activation='tanh', return_sequences=True,
                       go_backwards=True)
backward_layer2 = LSTM(14, activation='tanh', return_sequences=False,
                       go_backwards=True)
BiLSTM_model.add(Bidirectional(forward_layer1, backward_layer=backward_layer1))
#BiLSTM_model.add(LayerNormalization())
BiLSTM_model.add(Bidirectional(forward_layer2, backward_layer=backward_layer2))
#BiLSTM_model.add(LayerNormalization())
BiLSTM_model.add(Dense(myTrainY.shape[1], activation='linear'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
BiLSTM_model.compile(loss="huber_loss",
                     optimizer=adam,
                     metrics=["mean_squared_error"])

BiLSTM_model.load_weights('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/models/weightsBiLSTM_'+str(officeIDX)+'.hdf5')

In [93]:
LSTM_model = Sequential()
LSTM_model.add(Input(shape=(myTrainX.shape[1], myTrainX.shape[2])))
LSTM_model.add(LSTM(32, activation='sigmoid', return_sequences=True))
LSTM_model.add(LSTM(32, activation='sigmoid', return_sequences=False))
LSTM_model.add(Dense(myTrainY.shape[1], activation='linear'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
LSTM_model.compile(loss="huber_loss",
                   optimizer=adam,
                   metrics=["mean_squared_error"])
LSTM_model.load_weights('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/models/weightsLSTM_'+str(officeIDX)+'.hdf5')

In [94]:
RF_model = load(open('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/models/RFmodel_'+str(officeIDX)+'.pkl', 'rb'))

**Create Forecasts**

In [95]:
BiLSTM_forecast = BiLSTM_model.predict(myTestX)
print(BiLSTM_forecast.shape)
print('RMSE for BiLSTM Forecast on Test Set = ', calcInvScaledError(np.array(BiLSTM_forecast), np.array(testY),
                                                                    errorMetric='RMSE', scaler=scaler, dataToScalerIndices=[officeIDX]*len(BiLSTM_forecast)))

LSTM_forecast = LSTM_model.predict(myTestX)
print(LSTM_forecast.shape)
print('RMSE for LSTM Forecast on Test Set = ', calcInvScaledError(np.array(LSTM_forecast), np.array(testY),
                                                                  errorMetric='RMSE', scaler=scaler, dataToScalerIndices=[officeIDX]*len(LSTM_forecast)))

RF_forecast = RF_model.predict(testX)
print(RF_forecast.shape)
print('RMSE for Random Forest Forecast on Test Set = ', calcInvScaledError(np.array(RF_forecast), np.array(testY),
                                                                           errorMetric='RMSE', scaler=scaler, dataToScalerIndices=[officeIDX]*len(RF_forecast)))

(10978, 56)
RMSE for BiLSTM Forecast on Test Set =  68.24670867292984
(10978, 56)
RMSE for LSTM Forecast on Test Set =  54.295399616355645
(10978, 56)
RMSE for Random Forest Forecast on Test Set =  56.843559618420095


**Create Baseline Forecasts**

In [96]:
# Naive Method
def naive_forecast(data, readingsToForecast, naivePeriod=1):
    # Reduce dimensions of data if there are 3 dimensions in the data
    if len(data.shape)==3:
        data = data[:,:,0]
    if naivePeriod==1:
        return data[:, -readingsToForecast:]
    elif naivePeriod*readingsToForecast>data.shape[1]:
        raise Exception("naivePeriod*readingsToForecast must be smaller than or equal to the number of time-steps in the data")
    else:
        return data[:, -readingsToForecast*naivePeriod:-readingsToForecast*(naivePeriod-1)]
naiveForecastTrain = naive_forecast(myTrainX, 56, 1)
print(naiveForecastTrain.shape)
naiveForecastTest = naive_forecast(myTestX, 56, 1)
print(naiveForecastTest.shape)

# Average Method
def average_forecast(data, readingsToForecast):
    # Reduce dimensions of data if there are 3 dimensions in the data
    if len(data.shape)==3:
        data = data[:,:,0]
    temp = np.zeros((data.shape[0], readingsToForecast))
    counter = 0
    for i in range(data.shape[1], 0, -readingsToForecast):
        temp = temp + data[:, i-readingsToForecast:i]
        counter+=1
        #print(counter, i-readingsToForecast, i)
    temp = temp/counter
    return temp
averageForecastTrain = average_forecast(myTrainX, 56)
print(averageForecastTrain.shape)
averageForecastTest = average_forecast(myTestX, 56)
print(averageForecastTest.shape)

(46425, 56)
(10978, 56)
(46425, 56)
(10978, 56)


**Read SARIMA Forecast - and convert to windows**

In [97]:
SARIMA_fc = joblib.load(open('/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/data/arima_forecast_complete'+str(officeIDX)+'.pkl', 'rb'))

In [98]:
print(SARIMA_fc.shape)
SARIMA_forecast = np.zeros(np.array(testY).shape)

continuousSubSequenceStartIndex = 0
continuousSubSequenceStopIndex  = 0
searchingForNAN = True
tempCounter = 0
seqCounter = 0
while seqCounter<len(testScaled[officeIDX]):
    if seqCounter==len(testScaled[officeIDX])-1:
        if np.isnan(testScaled[officeIDX][seqCounter]):
            continuousSubSequenceStopIndex = seqCounter - 1
        else:
            continuousSubSequenceStopIndex = seqCounter
        ### Perform continuousSubSequence Operation - Extract Windows ###
        if continuousSubSequenceStopIndex - continuousSubSequenceStartIndex + 1 >= historySteps + leadSteps:
            for k in range(continuousSubSequenceStartIndex, continuousSubSequenceStopIndex+1):
                if k + historySteps + leadSteps <= continuousSubSequenceStopIndex + 1:
                    SARIMA_forecast[tempCounter, :] = SARIMA_fc[k+historySteps:k+historySteps+leadSteps]
                    tempCounter += 1
                    #dataX.append(testScaled[officeIDX][k:k+historySteps])
                    #dataY.append(testScaled[officeIDX][k+historySteps:k+historySteps+leadSteps])
        break
    if not np.isnan(testScaled[officeIDX][seqCounter]):
        if not searchingForNAN:
            searchingForNAN = True
            continuousSubSequenceStartIndex = seqCounter
    else:
        if searchingForNAN:
            searchingForNAN = False
            continuousSubSequenceStopIndex = seqCounter - 1
            ### Perform continuousSubSequence Operation - Extract Windows ###
            if continuousSubSequenceStopIndex - continuousSubSequenceStartIndex + 1 >= historySteps + leadSteps:
                for k in range(continuousSubSequenceStartIndex, continuousSubSequenceStopIndex+1):
                    if k + historySteps + leadSteps <= continuousSubSequenceStopIndex + 1:
                        SARIMA_forecast[tempCounter, :] = SARIMA_fc[k+historySteps:k+historySteps+leadSteps]
                        tempCounter += 1
                        #dataX.append(officesData[k:k+historySteps])
                        #dataY.append(officesData[k+historySteps:k+historySteps+leadSteps])
    seqCounter += 1

print(SARIMA_forecast.shape)

(11704,)
(10978, 56)


**Compare RMSE**

In [99]:
print('RMSE for SARIMA Method Forecast on Test Set = ', calcInvScaledError(np.array(SARIMA_forecast), np.array(testY),
                                                                           errorMetric='RMSE', scaler=scaler, dataToScalerIndices=[officeIDX]*len(averageForecastTest)))

print('RMSE for BiLSTM Forecast on Test Set = ', calcInvScaledError(np.array(BiLSTM_forecast), np.array(testY),
                                                                    errorMetric='RMSE', scaler=scaler, dataToScalerIndices=[officeIDX]*len(BiLSTM_forecast)))

print('RMSE for LSTM Forecast on Test Set = ', calcInvScaledError(np.array(LSTM_forecast), np.array(testY),
                                                                  errorMetric='RMSE', scaler=scaler, dataToScalerIndices=[officeIDX]*len(LSTM_forecast)))

print('RMSE for Random Forest Forecast on Test Set = ', calcInvScaledError(np.array(RF_forecast), np.array(testY),
                                                                           errorMetric='RMSE', scaler=scaler, dataToScalerIndices=[officeIDX]*len(RF_forecast)))

print('RMSE for Naive Method Forecast on Test Set = ', calcInvScaledError(np.array(naiveForecastTest), np.array(testY),
                                                                          errorMetric='RMSE', scaler=scaler, dataToScalerIndices=[officeIDX]*len(naiveForecastTest)))

print('RMSE for Average Method Forecast on Test Set = ', calcInvScaledError(np.array(averageForecastTest), np.array(testY),
                                                                            errorMetric='RMSE', scaler=scaler, dataToScalerIndices=[officeIDX]*len(averageForecastTest)))

RMSE for SARIMA Method Forecast on Test Set =  77.57328863134224
RMSE for BiLSTM Forecast on Test Set =  68.24670867292984
RMSE for LSTM Forecast on Test Set =  54.295399616355645
RMSE for Random Forest Forecast on Test Set =  56.843559618420095
RMSE for Naive Method Forecast on Test Set =  77.86956906835994
RMSE for Average Method Forecast on Test Set =  69.93668394794436


**Plot Results over Sample Window**

In [100]:
class MyFormatter(Formatter):
    def __init__(self, dates, fmt='%d-%m-%Y\n%H:%M:%S'):
        self.dates = dates
        self.fmt = fmt

    def __call__(self, x, pos=0):
        'Return the label for time x at position pos'
        ind = int(np.round(x))
        if ind >= len(self.dates) or ind < 0:
            return ''
        ts = (self.dates[ind] - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
        return datetime.utcfromtimestamp(ts).strftime(self.fmt)

In [109]:
# Window to Plot Results on
window_idx_to_forecast = 10702#65#60

# Create Pandas Series for Index Matching while Plotting
completeSeries = pd.Series(np.concatenate((testX[window_idx_to_forecast], testY[window_idx_to_forecast])))
seriesX = completeSeries[:len(testX[window_idx_to_forecast])]
seriesY = completeSeries[len(testX[window_idx_to_forecast]):]

completeDateSeries = pd.Series(np.concatenate((dateTestX[window_idx_to_forecast], dateTestY[window_idx_to_forecast])))
dateSeriesX = completeDateSeries[:len(dateTestX[window_idx_to_forecast])]
dateSeriesY = completeDateSeries[len(dateTestX[window_idx_to_forecast]):]

In [110]:
includeARIMA = False
if includeARIMA:
    arima_model = ARIMA(seriesX, order=(1,1,2), seasonal_order=(1,0,2,56))
    arima_modelFit = arima_model.fit()
    arima_forecast = arima_modelFit.get_forecast(leadSteps, alpha=0.05) # 95% confidence interval

In [111]:
# Make as pandas series

SARIMA_forecast_series = pd.Series(SARIMA_forecast[window_idx_to_forecast], index=dateSeriesY.index)
SARIMA_forecast_seriesTT = pd.Series(invertStandardization(scaler, np.array([SARIMA_forecast_series]), np.array([officeIDX]))[0], index=dateSeriesY.index)

BiLSTM_forecast_series = pd.Series(BiLSTM_forecast[window_idx_to_forecast], index=dateSeriesY.index)
BiLSTM_forecast_seriesTT = pd.Series(invertStandardization(scaler, np.array([BiLSTM_forecast_series]), np.array([officeIDX]))[0], index=dateSeriesY.index)

LSTM_forecast_series = pd.Series(LSTM_forecast[window_idx_to_forecast], index=dateSeriesY.index)
LSTM_forecast_seriesTT = pd.Series(invertStandardization(scaler, np.array([LSTM_forecast_series]), np.array([officeIDX]))[0], index=dateSeriesY.index)

RF_forecast_series = pd.Series(RF_forecast[window_idx_to_forecast], index=dateSeriesY.index)
RF_forecast_seriesTT = pd.Series(invertStandardization(scaler, np.array([RF_forecast_series]), np.array([officeIDX]))[0], index=dateSeriesY.index)

seriesXTT = pd.Series(invertStandardization(scaler, np.array([seriesX]), np.array([officeIDX]))[0], index=dateSeriesX.index)
seriesYTT = pd.Series(invertStandardization(scaler, np.array([seriesY]), np.array([officeIDX]))[0], index=dateSeriesY.index)

naive_forecast_series = pd.Series(naiveForecastTest[window_idx_to_forecast], index=dateSeriesY.index)
naive_forecast_seriesTT = pd.Series(invertStandardization(scaler, np.array([naive_forecast_series]), np.array([officeIDX]))[0], index=dateSeriesY.index)

average_forecast_series = pd.Series(averageForecastTest[window_idx_to_forecast], index=dateSeriesY.index)
average_forecast_seriesTT = pd.Series(invertStandardization(scaler, np.array([average_forecast_series]), np.array([officeIDX]))[0], index=dateSeriesY.index)

if includeARIMA:
    arima_fc = arima_forecast.predicted_mean
    arima_se = arima_forecast.se_mean
    arima_conf = arima_forecast.conf_int()

    # Make as pandas series
    arima_fc_series = pd.Series(arima_fc, index=dateSeriesY.index)
    arima_lower_series = pd.Series(arima_conf['lower y'], index=dateSeriesY.index)
    arima_upper_series = pd.Series(arima_conf['upper y'], index=dateSeriesY.index)
    arima_fc_seriesTT = pd.Series(invertStandardization(scaler, np.array([arima_fc_series]), np.array([officeIDX]))[0], index=dateSeriesY.index)
    arima_lower_seriesTT = pd.Series(invertStandardization(scaler, np.array([arima_lower_series]), np.array([officeIDX]))[0], index=dateSeriesY.index)
    arima_upper_seriesTT = pd.Series(invertStandardization(scaler, np.array([arima_upper_series]), np.array([officeIDX]))[0], index=dateSeriesY.index)


# Plot
formatter = MyFormatter(np.append(dateTestX[window_idx_to_forecast], dateTestY[window_idx_to_forecast]))
fig, ax = plt.subplots(figsize=(30, 9))
ax.xaxis.set_major_formatter(formatter)
plt.rc('font', size=17)         # controls default text sizes
plt.rc('axes', titlesize=30)    # fontsize of the axes title
plt.rc('axes', labelsize=30)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=26)   # fontsize of the tick labels
plt.rc('ytick', labelsize=26)   # fontsize of the tick labels
plt.rc('legend', fontsize=28)   # legend fontsize
plt.rc('figure', titlesize=35)  # fontsize of the figure title
plt.plot(seriesXTT, label='Historical Data', color='blue', marker='', linestyle='-', linewidth=3, markersize=2)
plt.plot(seriesYTT, label='Actual Data in Future', color='blue', marker='', linestyle='--', linewidth=3, markersize=2)
plt.plot(SARIMA_forecast_seriesTT, label='SARIMA Forecast', color='olive', marker='', linestyle='-', linewidth=3, markersize=2)
plt.plot(RF_forecast_seriesTT, label='Random Forest Forecast', color='green', marker='', linestyle='-', linewidth=3, markersize=2)
plt.plot(BiLSTM_forecast_seriesTT, label='Bi-LSTM Forecast', color='red', marker='', linestyle='-', linewidth=3, markersize=2)
plt.plot(LSTM_forecast_seriesTT, label='LSTM Forecast', color='magenta', marker='', linestyle='-', linewidth=3, markersize=2)
plt.plot(naive_forecast_seriesTT, label='Naive Forecast', color='slategray', marker='', linestyle='-', linewidth=3, markersize=2)
plt.plot(average_forecast_seriesTT, label='Average Forecast', color='tan', marker='', linestyle='-', linewidth=3, markersize=2)
if includeARIMA:
    plt.plot(arima_fc_seriesTT, label='ARIMA Forecast', color='rosybrown', marker='', linestyle='-', linewidth=3, markersize=2)
    plt.fill_between(arima_lower_seriesTT.index, arima_lower_seriesTT, arima_upper_seriesTT, 
                    color='gainsboro', alpha=.15, label='95% Confidence Interval (ARIMA)')
plt.xlabel("Timestamp", fontsize=30)
plt.ylabel("Electricity Consumption (in kW)", fontsize=30)
#plt.title('Forecast vs Actuals')
plt.legend(loc='lower left', fontsize=28)
plt.gca().set_ylim(bottom=50)
plt.rc('xtick', labelsize=26)   # fontsize of the tick labels
plt.rc('ytick', labelsize=26)   # fontsize of the tick labels
plt.savefig("/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/Results_"+str(officeIDX)+"_"+str(window_idx_to_forecast)+".pdf",
            bbox_inches = 'tight', pad_inches = 0.05)
plt.close()

In [ ]:
# Plot Model Architecture
'''
architecture_file = '/content/drive/My Drive/Colab Notebooks/Electricity Forecasting/modelArchitecture.pdf'
tf.keras.utils.plot_model(model, to_file=architecture_file, show_shapes=True)
'''